# Welcome to OSMNxMapping ☀️!

_Chapter 5_ will be about playing with a new parameter, the `target` parameter in OSMNxMapping. `Target` will primarily be useful when you do not want analysis to the edges' level but to the nodes' one or both nodes and edges.

We'll process PLUTO building data from New York City, enriching a Manhattan street network with the average number of floors and experimenting with targeting nodes, edges, or both during enrichment and visualisation. This example demonstrates how `target` influences where data is enriched and how it’s displayed, offering flexibility in urban analysis workflows.

**Goals**:
- Import the OSMNxMapping library and necessary modules.
- Initialise an OSMNxMapping instance.
- Build a step-by-step workflow to load, preprocess, map, enrich, and visualise data.
- Experiment with the `target` parameter in enrichment (`nodes`, `edges`, or `both`).
- Visualise results with different `target` settings (`edges` and `both`).
- Understand the impact of `target` on enrichment and visualisation outputs.

This notebook uses local data (e.g., CSV format) without Auctus, focusing on a sample `pluto.csv` file. For simpler workflows, see chapter 1's notebook.

Let’s get started! 🚀

## Step 1: Import the Library and Modules

We import the `osmnx_mapping` library as `oxm`, along with `osmnx` and `geopandas` for network and geospatial operations. These modules support our pipeline steps.

In [ ]:
import osmnx_mapping as oxm
import osmnx as ox
import geopandas as gpd

## Step 2: Initialise an OSMNxMapping Instance

We create an `OSMNxMapping` instance named `mapping`. This instance manages our urban data processing workflow.

In [ ]:
mapping = oxm.OSMNxMapping()

## Step 3: Load the PLUTO Data

We load PLUTO building data from a CSV file using the `loader` mixin, specifying latitude and longitude columns.

> **Note**: Adjust the file path (`../data/PLUTO/csv/pluto.csv`) and ensure column names match your dataset.

In [ ]:
data = mapping.loader.load_from_file(
    "../data/PLUTO/csv/pluto.csv", latitude_column="latitude", longitude_column="longitude"
)

## Step 4: Query the Manhattan Street Network

Using the `road_networks` mixin, we fetch a drive network for Manhattan, returning the graph, nodes, and edges.

In [ ]:
graph, nodes, edges = mapping.road_networks.network_from_place("Manhattan, New York City, USA")

## Step 5: Impute Missing Coordinates

We apply `SimpleGeoImputer` to drop rows with missing coordinates, ensuring clean data for mapping.

In [ ]:
data = (
    mapping.preprocessing
    .with_default_imputer(latitude_column_name="latitude", longitude_column_name="longitude")
    .transform(input_data=data)
)

## Step 6: Filter Data by Bounding Box

We use `CreatePreprocessor` with a `BoundingBoxFilter` to retain only data within the network’s spatial bounds.

In [ ]:
from osmnx_mapping.modules.preprocessing import CreatePreprocessor

data = (
    CreatePreprocessor().with_filter(
        filter_type="BoundingBoxFilter",
        nodes=nodes
    ).build().transform(data)
)

## Step 7: Map Data to Nearest Nodes

We map building coordinates to the nearest street nodes using the `road_networks` mixin, storing results in `nearest_node`.

In [ ]:
mapped_data = mapping.road_networks.map_nearest_street(
    data, longitude_column="longitude", latitude_column="latitude"
)

## Step 8: Enrich the Network with Target Exploration

We create an enricher to compute the average number of floors per node, targeting `both` nodes and edges. The `target` parameter (`nodes`, `edges`, or `both`) controls where enrichment is applied. We preview the configuration before building.

> **Experiment**: Using `target="both"` enriches both nodes and edges, allowing us to test its effect in visualisation.

In [ ]:
enricher = (
    oxm.CreateEnricher()
    .with_data(group_by="nearest_node", values_from="numfloors")
    .aggregate_with(method="mean", target="both", output_column="avg_floors")
)
print(enricher.preview())

enricher = enricher.build()
_, enriched_graph, enriched_nodes, enriched_edges = enricher.enrich(
    mapped_data, graph, nodes, edges
)

## Step 9: Visualise with Target="edges"

We use `StaticVisualiser` with `target="edges"` to focus on street segments, displaying the average floor counts statically.

In [ ]:
fig = mapping.visual(visualiser=oxm.StaticVisualiser()).visualise(
    enriched_graph, enriched_nodes, enriched_edges, result_columns="avg_floors", target="edges"
)
fig

## Step 9: Visualise with Target="Nodes"

We use `InteractiveVisualiser` with `target="Nodes"` to display both Nodes interactively, showing the combined enrichment effect.

> **Note**: Install Jupyter extensions for interactive visualisation (see README).

In [ ]:
fig = mapping.visual(visualiser=oxm.InteractiveVisualiser()).visualise(
    enriched_graph, enriched_nodes, enriched_edges, result_columns="avg_floors", target="nodes"
)
fig

## Step 11: Visualise with Target="both"

We use `InteractiveVisualiser` with `target="both"` to display both nodes and edges interactively, showing the combined enrichment effect.

> **Note**: Install Jupyter extensions for interactive visualisation (see README).

In [ ]:
fig = mapping.visual(visualiser=oxm.InteractiveVisualiser()).visualise(
    enriched_graph, enriched_nodes, enriched_edges, result_columns="avg_floors", target="both"
)
fig

## Conclusion

Congratulations! 🎉 You’ve explored the `target` parameter with 'Advanced Pipelining [Exploring Targets]' in OSMNxMapping. By enriching a Manhattan network with PLUTO data and visualising it with `target="edges"` and `target="both"`, you’ve seen how it controls enrichment placement and visualisation focus. This flexibility enhances urban analysis precision.

For simpler workflows, see 'Basic Pipelining'. For multiple mappings or enrichments, check 'Advanced Pipelining [Multiple Mappings and Enrichments]' or 'Advanced Pipelining [Multiple Enrichments]'. Dive into the OSMNxMapping API docs and `examples/` for more!

Happy urban mapping! 🌆